## Introduction

In this tutorial, we will model and simulate a **Steel Plant** by connecting various components like the **Electrolyser**, **DRI Plant**, and **Electric Arc Furnace (EAF)**. We will also demonstrate how to incorporate **DSM Load Shifting** to optimize flexibility in energy consumption.

# Steel Plant Simulation Tutorial

In this tutorial, we will model and simulate a **Steel Plant** by connecting various components like the **Electrolyser**, **DRI Plant**, and **Electric Arc Furnace (EAF)**. We will also demonstrate how to incorporate **DSM Load Shifting** to optimize flexibility in energy consumption.

---

## Step 1: Cloning and Setting Up the ASSUME Framework

To start, you need to clone the ASSUME framework from the official GitHub repository. Instructions can also be found in the [ASSUME documentation](https://assume.readthedocs.io/en/latest/installation.html).


In [ ]:
!pip install assume-framework
!git clone https://github.com/assume-framework/assume.git

: 

## Setting Up the Simulation Environment

In this notebook, we will walk through the process of setting up and simulating a **Steel Plant** using the ASSUME framework. This steel plant will connect components such as an **Electrolyser**, **DRI Plant**, and an **Electric Arc Furnace (EAF)** to optimize electricity usage.

### Objective:
Our goal is to create a steel plant model that minimizes the cost of electricity usage by simulating the demand and integrating flexibility using load-shifting techniques.

---

### Step 1: Cloning the ASSUME Framework

Before proceeding with the steel plant setup, we first need to **clone** and install the ASSUME framework, which contains the necessary modules for simulation. The installation guide can be found here: [ASSUME Documentation](https://assume.readthedocs.io/en/latest/installation.html).

If the ASSUME framework is already installed on your system, you can skip this step.

> **Note**: If you already have the framework installed, you can skip this step.

Once installed, you can proceed with setting up your **Steel Plant Simulation**.

---

## Step 2: Setting Up the Steel Plant

Once the ASSUME framework is installed, we will now set up the **Steel Plant** by defining its components and their properties. The steel plant consists of three primary components:

- **Electrolyser**: Used to produce hydrogen for steel production.
- **DRI Plant**: Directly reduces iron ore using hydrogen.
- **Electric Arc Furnace (EAF)**: Converts the reduced iron into steel.

The objective of this step is to define these components, along with their operational constraints (such as power, efficiency, ramp rates), and then create an instance of the **Steel Plant**.



In [ ]:
from dst_components import create_electrolyser, create_driplant, create_electric_arc_furnance
from steel_plant import SteelPlant
import pandas as pd

# Define time steps for the simulation
time_steps = pd.date_range(start='2023-01-01', periods=24, freq='H')

# Define components for the steel plant
components = {
    'electrolyser': {
        'rated_power': 50,  # MW
        'min_power': 10,    # MW
        'ramp_up': 5,       # MW/hr
        'ramp_down': 5,     # MW/hr
        'min_operating_time': 1,  # hr
        'min_down_time': 1,  # hr
        'efficiency': 0.7,   # fraction
    },
    'dri_plant': {
        'specific_hydrogen_consumption': 0.01,  # MWh per ton of DRI
        'specific_natural_gas_consumption': 0.1, # MWh per ton of DRI
        'specific_electricity_consumption': 0.05, # MWh per ton of DRI
        'specific_iron_ore_consumption': 1,  # ton per ton of DRI
        'rated_power': 100,  # MW
        'min_power': 30,     # MW
        'fuel_type': 'both', 
        'ramp_up': 10,       # MW/hr
        'ramp_down': 10,     # MW/hr
        'min_operating_time': 2,  # hr
        'min_down_time': 2,   # hr
    },
    'eaf': {
        'rated_power': 120,  # MW
        'min_power': 40,     # MW
        'specific_electricity_consumption': 0.12, # MWh per ton of steel
        'specific_dri_demand': 1,  # ton per ton of steel
        'specific_lime_demand': 0.1, # ton per ton of steel
        'ramp_up': 8,        # MW/hr
        'ramp_down': 8,      # MW/hr
        'min_operating_time': 3,  # hr
        'min_down_time': 2,   # hr
    }
}

# Create a Steel Plant instance
steel_plant = SteelPlant(
    id='steel_plant_001',
    unit_operator='SteelCo',
    bidding_strategies={'EOM': 'NaiveDASteelplantStrategy'},
    components=components,
    objective='min_variable_cost',
    demand=1000,  # tons of steel to produce
    cost_tolerance=15,  # flexibility cost tolerance
    index=time_steps
)


## Step 3: Connecting the Components

After setting up the steel plant components, we will connect them to create a process flow for steel production. The **Electrolyser** produces hydrogen, which is consumed by the **DRI Plant**. The DRI produced by the DRI Plant is then processed by the **Electric Arc Furnace (EAF)**.

In this step, we will call the `initialize_process_sequence()` function, which links these components together.


In [ ]:
# Initialize the process sequence for the steel plant
steel_plant.initialize_process_sequence()

## Step 4: Simulating Steel Plant Operation

Now that the steel plant is set up and the components are connected, we can simulate its operation. The goal is to minimize variable costs by optimizing electricity consumption.

We will call the `determine_optimal_operation_without_flex()` method to calculate the optimal operation without load-shifting.

In [ ]:
# Simulate steel plant operation to minimize costs
steel_plant.determine_optimal_operation_without_flex()

# Get the optimal power requirement and total cost
optimal_power = steel_plant.opt_power_requirement
total_cost = steel_plant.total_cost

print("Optimal Power Requirement (MW):")
print(optimal_power)

print("Total Cost of Operation:")
print(total_cost)

## Step 5: Integrating DSM Load Shifting

The next step is to integrate **load-shifting** capabilities using the **DSM Load Shift** module. This allows the steel plant to adjust its power consumption based on market conditions, maximizing flexibility.

We will now switch to `flexibility_measure = 'max_load_shift'` and recalculate the optimal operation with flexibility.

In [ ]:
# Enable DSM load shifting by adjusting the flexibility measure
steel_plant.flexibility_measure = 'max_load_shift'

# Simulate the steel plant with load-shifting flexibility
steel_plant.determine_optimal_operation_with_flex()

# Get the adjusted power requirement after load shifting
flex_power = steel_plant.flex_power_requirement

print("Power Requirement After Load Shifting (MW):")
print(flex_power)

# Show total operational cost after flexibility is applied
print("Total Cost After Load Shifting:")
print(steel_plant.total_cost)

## Step 6: Visualizing the Results

Finally, we can visualize the difference in power consumption before and after load-shifting. This helps us understand the impact of flexibility on electricity usage and cost.

In [ ]:
import matplotlib.pyplot as plt

# Plot power requirement before and after load shifting
plt.plot(optimal_power, label="Optimal Power (No Flex)")
plt.plot(flex_power, label="Power with Load Shift", linestyle="--")
plt.title("Steel Plant Power Requirement with and without Flexibility")
plt.xlabel("Time")
plt.ylabel("Power (MW)")
plt.legend()
plt.show()

## Conclusion

In this notebook, we demonstrated the setup and simulation of a **Steel Plant** using the ASSUME framework. We created components, connected them, and simulated the plant’s operation both with and without load-shifting flexibility.

Key takeaways:
- The steel plant can minimize electricity costs by optimizing its power consumption.
- Integrating DSM Load Shifting allows the plant to respond to market conditions and increase its operational flexibility.

This tutorial provides a foundation for simulating and optimizing energy usage in industrial processes.